In [4]:
import requests
from bs4 import BeautifulSoup
import csv

def fetch_news(url):
    # 發送請求並解析 HTML
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # 選擇包含新聞的元素
    articles = soup.find_all('article', class_='IFHyqb')

    # 提取標題、鏈接和時間
    news_list = []
    for article in articles:
        title_element = article.find('a', class_='JtKRv')
        title = title_element.get_text(strip=True) if title_element else '未知'
        link = title_element.get('href', '').strip() if title_element else ''
        full_link = requests.compat.urljoin(url, link)

        # 獲取新聞來源
        news_source = article.find('div', class_='vr1PYe')
        source_name = news_source.get_text(strip=True) if news_source else '未知'
        
        # 獲取新聞時間
        time_element = article.find('div', class_='UOVeFe').find('time', class_='hvbAAd') if article.find('div', class_='UOVeFe') else None
        date = time_element.get_text(strip=True) if time_element else '未知'
        
        news_list.append({
            '標題': title,
            '連結': full_link,
            '來源': source_name,
            '時間': date
        })

    return news_list

def save_to_csv(news_items, filename):
    # 定義CSV檔案的標頭
    headers = ['標題', '連結', '來源', '時間']
    
    # 打開CSV檔案並寫入資料
    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=headers)
        
        # 寫入標頭
        writer.writeheader()
        
        # 寫入新聞項目
        for item in news_items:
            writer.writerow(item)

# 多個網址
urls = [
    'https://news.google.com/search?q=%E5%9C%8B%E9%9A%9B%E5%A4%A7%E9%9B%A8%20when%3A30d&hl=zh-TW&gl=TW&ceid=TW%3Azh-Hant',#國際大雨 when:30d
    'https://news.google.com/search?q=%E5%9C%8B%E9%9A%9B%E8%B1%AA%E9%9B%A8%20when%3A30d&hl=zh-TW&gl=TW&ceid=TW%3Azh-Hant',#國際豪雨 when:30d
    'https://news.google.com/search?q=%E5%9C%8B%E9%9A%9B%E6%9A%B4%E9%9B%A8%20when%3A30d&hl=zh-TW&gl=TW&ceid=TW%3Azh-Hant',#國際暴雨 when:30d
    'https://news.google.com/search?q=%E5%9C%8B%E9%9A%9B%E6%B7%B9%E6%B0%B4%20when%3A30d&hl=zh-TW&gl=TW&ceid=TW%3Azh-Hant',#國際淹水 when:30d
    'https://news.google.com/search?q=%E5%9C%8B%E9%9A%9B%E6%B4%AA%E6%B0%B4%20when%3A30d&hl=zh-TW&gl=TW&ceid=TW%3Azh-Hant',#國際洪水 when:30d
    'https://news.google.com/search?q=%E5%9C%8B%E9%9A%9B%E6%B0%B4%E7%81%BD%20when%3A30d&hl=zh-TW&gl=TW&ceid=TW%3Azh-Hant',#國際水災 when:30d
    'https://news.google.com/search?q=%E5%9C%8B%E9%9A%9B%E9%A2%B1%E9%A2%A8%20when%3A30d&hl=zh-TW&gl=TW&ceid=TW%3Azh-Hant',#國際颱風 when:30d
    'https://news.google.com/search?q=%E5%9C%8B%E9%9A%9B%E9%A2%B6%E9%A2%A8%20when%3A30d&hl=zh-TW&gl=TW&ceid=TW%3Azh-Hant',#國際颶風 when:30d
    'https://news.google.com/search?q=%E5%9C%8B%E9%9A%9B%E9%A2%A8%E7%81%BD%20when%3A30d&hl=zh-TW&gl=TW&ceid=TW%3Azh-Hant',#國際風災 when:30d
    'https://news.google.com/search?q=%E5%9C%8B%E9%9A%9B%E6%B5%B7%E5%98%AF%20when%3A30d&hl=zh-TW&gl=TW&ceid=TW%3Azh-Hant',#國際海嘯 when:30d
    'https://news.google.com/search?q=%E5%9C%8B%E9%9A%9B%E5%9C%B0%E9%9C%87%20when%3A30d&hl=zh-TW&gl=TW&ceid=TW%3Azh-Hant',#國際地震 when:30d
    'https://news.google.com/search?q=%E5%9C%8B%E9%9A%9B%E4%B9%BE%E6%97%B1%20when%3A30d&hl=zh-TW&gl=TW&ceid=TW%3Azh-Hant',#國際乾旱 when:30d
    'https://news.google.com/search?q=%E5%9C%8B%E9%9A%9B%E6%97%B1%E7%81%BD%20when%3A30d&hl=zh-TW&gl=TW&ceid=TW%3Azh-Hant'#國際旱災 when:30d
]

# 爬取所有網址的新聞
all_news_items = []
for url in urls:
    news_items = fetch_news(url)
    all_news_items.extend(news_items)

# 保存所有新聞到 CSV 檔案
filename = '123all_news.csv'
save_to_csv(all_news_items, filename)

print(f"所有新聞已保存到 {filename}")

import pandas as pd
import sqlite3

# CSV 檔案路徑
csv_file_path = 'all_news.csv'

# 讀取 CSV 檔案
df = pd.read_csv(csv_file_path)

# 連接到 SQLite 資料庫（如果資料庫不存在，會自動創建）
conn = sqlite3.connect('123news.db')

# 確保資料庫中有正確的表結構
# 刪除舊的資料表（如果存在）
conn.execute('DROP TABLE IF EXISTS news_item')

# 創建新的資料表
create_table_query = '''
CREATE TABLE news_item (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT,
    url TEXT,
    source TEXT,
    date TEXT
)
'''
conn.execute(create_table_query)

# 將 DataFrame 中的資料插入 SQLite
for index, row in df.iterrows():
    title = row['標題']
    url = row['連結']
    source = row['來源']
    date_str = row['時間']  # 保持 CSV 中的時間格式不變

    # 插入資料到 SQLite
    conn.execute('''
        INSERT INTO news_item (title, url, source, date) VALUES (?, ?, ?, ?)
    ''', (title, url, source, date_str))

# 提交變更並關閉連接
conn.commit()
conn.close()

print('CSV 資料已成功儲存到 SQLite 資料庫。')





所有新聞已保存到 123all_news.csv
CSV 資料已成功儲存到 SQLite 資料庫。


In [9]:
import pandas as pd

# 讀取 CSV 檔案
df = pd.read_csv('123all_news.csv')

# 定義有效的來源
valid_sources = {'經濟日報', 'Yahoo奇摩新聞', 'Newtalk新聞', '光明日报'}

# 選擇來源在有效列表中的行
filtered_df = df[df['來源'].isin(valid_sources)]

# 只保留 '連結' 和 '來源' 欄位
filtered_df = filtered_df[['連結', '來源']]

# 將篩選後的內容儲存到新的 CSV 檔案
filtered_df.to_csv('filtered_news.csv', index=False, encoding='utf-8')

print('提取後的數據已儲存到 filtered_news.csv')


提取後的數據已儲存到 filtered_news.csv


In [14]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# 設定 User-Agent
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# 定義統一抓取內容的函數
def fetch_article_content(source_name, url):
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    if source_name == '經濟日報':
        article_body = soup.find('section', class_='article-body__editor')
        if article_body:
            paragraphs = article_body.find_all('p')
            return '\n'.join(p.get_text() for p in paragraphs)
    
    elif source_name == 'Yahoo奇摩新聞':
        caas_body_div = soup.find('div', class_='caas-body')
        if caas_body_div:
            paragraphs = caas_body_div.find_all('p')
            return '\n'.join(p.get_text() for p in paragraphs)
    
    elif source_name == 'Newtalk新聞':
        article_div = soup.find('div', class_='articleBody clearfix')
        if article_div:
            paragraphs = article_div.find_all('p')
            return '\n'.join(p.get_text() for p in paragraphs)
    
    # 可以在這裡添加其他來源的處理邏輯
    return '未找到內容'

# 讀取 CSV 文件
df = pd.read_csv('filtered_news.csv')

# 初始化一個列表來儲存爬取的結果
results = []

# 逐筆處理每個條目
for _, row in df.iterrows():
    source_name = row['來源']
    url = row['連結']
    content = fetch_article_content(source_name, url)
    
    results.append({
        '連結': url,
        '來源': source_name,
        '內容': content
    })

# 輸出結果到控制台
for result in results:
    print(f"連結: {result['連結']}")
    print(f"來源: {result['來源']}")
    print(f"內容: {result['內容'][:500]}...")  # 限制內容長度以避免輸出過長
    print('-' * 80)

print('爬取後的內容已輸出到控制台')


連結: https://news.google.com/read/CBMi2ANBVV95cUxOR2FZSWExYmNZUG8za2ItQy1WU1V3RndnVWoxYUNnSGpwd0ptLW1RcmIzMnY0Yno0aTcybmFUZkdCM3NIa2RGckx4RUZDU01uUVp2dlFaZVdibk9hUkMyanNMcG1QUkZZQ3FvR2h3RWhjTXRTMlRtZ0Yzb2pzbkVRYWRySEJoSG9hSWNfX2ZTY3E3T2hNYk02VDJNeFRCeEl4VUtCRTJZaTNDMmFELXkxY0E0SzNuSGdtMTRPVDkyYjd0eXRYRHE3SVNZWUN5T2xvN05WVHlWR3BibjAzVHFMWTBZN3ZWR1VmcWFaVVZFbVI5UlFIRWUwc0hraTdocC0ycGRZTy05dlU2Sktsb2p5Q0g2SFFFU28tODhuV3l3ZnFsX3BiaHlxYmdLYjQxNWtxYUUzenktb1pkSkEzOXlDbTdOdUtqS3I3QnFYQXh5dUNnZC1IdzJIeEtlUW1XU2xrSnZhZGRUMjNxdmxFRk5nby1zTkRZYVpkMUFnaXl1YjY4M2lHWjVqamtTeEJISExPRkRVUlVZYzBqUkhYTXBqMWp5dEVZeFNGajNEMlhoTDlLemFscHhjWjVsSXQyYms5cUZYbVhFNGVLZ2lrY2htN0J0WXpJY3p0?hl=zh-TW&gl=TW&ceid=TW%3Azh-Hant
來源: Yahoo奇摩新聞
內容: 未找到內容...
--------------------------------------------------------------------------------
連結: https://news.google.com/read/CBMiWkFVX3lxTFAzQXVCRTNXUExTWUx0empvYTVpb2l5VzBpVk9TSGZLQl9WNnRfS3M0cUF0UXNIZGg0dzBpb3ZsZDNENHM0b0poNV9zcUZnendrZ3lyS0pYU0d6Z9IBX0FVX3lxTE9xVXpPdjM0R2d1LTE5

In [13]:
import requests
from bs4 import BeautifulSoup

# 設定 User-Agent
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

url = 'https://newtalk.tw/news/view/2024-07-27/929819'
response = requests.get(url, headers=headers)
    
soup = BeautifulSoup(response.text, 'html.parser')
article_div = soup.find('div', class_='articleBody clearfix')
if article_div:
    paragraphs = article_div.find_all('p')
    content = '\n'.join(p.get_text() for p in paragraphs)
    print(content)
else:
    print('未找到內容')


中央氣象署發布豪雨特報，氣象署表示，受到低壓帶影響，易有短延時強降雨，今(27)日台東縣有局部大雨或豪雨，苗栗、花蓮、蘭嶼、綠島、南部地區及新竹、台中、南投山區有局部大雨發生的機率。
氣象署提醒，民眾應注意雷擊及強陣風，連日降雨，請注意坍方、落石及土石流，沿海低窪地區請慎防淹水，民眾應避免進入山區及河川活動。
